In [20]:
# Базовые библиотеки для работы с данными и вычислений
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Инструменты для ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report, confusion_matrix
)

# Визуализация
import seaborn as sns
import matplotlib.pyplot as plt

# MLflow и логирование
import mlflow
import mlflow.tensorflow
from mlflow.models.signature import infer_signature
import dagshub
from mlflow.exceptions import MlflowException

# Системные библиотеки
import time
import os
import tempfile
import datetime

In [21]:
def prepare_data(filepath='water_potability.csv'):
    """
    Подготовка данных из датасета о качестве воды

    Параметры:
    filepath (str): Путь к файлу с данными

    Возвращает:
    tuple: (X, y, feature_names)
        - X: матрица признаков
        - y: целевая переменная
        - feature_names: список имен признаков
    """
    # Загрузка данных
    df = pd.read_csv(filepath)

    # Создал две bias метрики и заменил пропуски нулями
    #df['sulfates_missing'] = df['Sulfate'].isna().astype(int)
    #df['ph_missing'] = df['ph'].isna().astype(int)


    #df['Sulfate'] = df['Sulfate'].fillna(df['Sulfate'].median())
    #df['ph'] = df['ph'].fillna(df['ph'].median())


    # Разделение на признаки и целевую переменную
    X = df.drop('Potability', axis=1)
    y = df['Potability']

    # Сохранение названий признаков для последующего использования
    feature_names = X.columns.tolist()

    return X, y, feature_names

In [22]:
def create_model(input_dim):
    """
    Создание архитектуры нейронной сети

    Параметры:
    input_dim (int): Размерность входного слоя

    Возвращает:
    keras.Model: Скомпилированная модель нейронной сети
    """
    # Создаем модель, используя функциональный API
    inputs = keras.Input(shape=(input_dim,))

    # Определяем слои
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(16, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    # Создаем модель
    model = keras.Model(inputs=inputs, outputs=outputs, name='water_quality_model')

    # Компиляция модели
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [23]:
def train_and_evaluate_model(X, y, epochs=50, batch_size=32):
    """
    Обучает модель и возвращает результаты обучения и оценки

    Параметры:
    X: матрица признаков
    y: целевая переменная
    epochs (int): количество эпох обучения
    batch_size (int): размер батча

    Возвращает:
    dict: Словарь с результатами обучения и оценки
    """
    # Разделение данных на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Стандартизация признаков
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Создание модели
    model = create_model(input_dim=X.shape[1])

    # Настройка ранней остановки для предотвращения переобучения
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    # Замер времени обучения
    start_time = time.time()
    # Обучение модели
    history = model.fit(
        X_train_scaled,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=2  # Изменено на 2 для более краткого вывода
    )
    training_time = time.time() - start_time

    # Получение предсказаний и замер времени предсказания
    start_pred_time = time.time()
    y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)
    y_pred_proba = model.predict(X_test_scaled)
    prediction_time = time.time() - start_pred_time

    # Расчет метрик качества модели
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average='weighted', zero_division=1),
        "recall": recall_score(y_test, y_pred, average='weighted', zero_division=1),
        "f1": f1_score(y_test, y_pred, average='weighted', zero_division=1),
        "roc_auc": roc_auc_score(y_test, y_pred_proba),
        "training_time": training_time,
        "prediction_time": prediction_time
    }

    # Формирование полного набора результатов
    evaluation_results = {
        'model': model,
        'scaler': scaler,
        'history': history,
        'metrics': metrics,
        'predictions': {
            'y_test': y_test,
            'y_pred': y_pred,
            'y_pred_proba': y_pred_proba
        },
        'data': {
            'X_test_scaled': X_test_scaled
        }
    }

    return evaluation_results

In [24]:
def log_to_mlflow(evaluation_results, experiment_name="Deep Learning Binary Classification", run_name=None):
    """
    Логирует результаты эксперимента в MLflow

    Параметры:
    evaluation_results (dict): Результаты обучения и оценки модели
    experiment_name (str): Название эксперимента
    run_name (str): Название запуска (если не указано, генерируется автоматически)
    """
    try:
        experiment_id = mlflow.create_experiment(experiment_name)
    except mlflow.exceptions.MlflowException:
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

    with mlflow.start_run(experiment_id=experiment_id, run_name=run_name):


        # Логирование тегов эксперимента
        mlflow.set_tag("model_type", "keras_deep_learning")
        mlflow.set_tag("data_version", "v1.0")

        # Извлечение данных из результатов
        model = evaluation_results['model']
        history = evaluation_results['history']
        metrics = evaluation_results['metrics']
        y_test = evaluation_results['predictions']['y_test']
        y_pred = evaluation_results['predictions']['y_pred']
        X_test_scaled = evaluation_results['data']['X_test_scaled']

        # Логирование параметров модели
        model_params = {
            "input_dim": X_test_scaled.shape[1],
            "optimizer": model.optimizer.get_config()["name"],
            "loss": model.loss,
            "metrics": model.metrics_names,
        }
        mlflow.log_params(model_params)

        # Логирование метрик
        for metric_name, metric_value in metrics.items():
            mlflow.log_metric(metric_name, metric_value)

        # Создание временной директории для артефактов
        temp_dir = tempfile.mkdtemp()

        # Сохранение и логирование отчета о классификации
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=1)
        report_df = pd.DataFrame(report).transpose()
        report_path = os.path.join(temp_dir, "classification_report.csv")
        report_df.to_csv(report_path)
        mlflow.log_artifact(report_path, "reports")

        # Создание и сохранение матрицы ошибок
        plt.figure(figsize=(8, 6))
        cm = confusion_matrix(y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        cm_path = os.path.join(temp_dir, "confusion_matrix.png")
        plt.savefig(cm_path)
        mlflow.log_artifact(cm_path, "plots")
        plt.close()

        # Сохранение истории обучения
        history_df = pd.DataFrame(history.history)
        history_path = os.path.join(temp_dir, "training_history.csv")
        history_df.to_csv(history_path)
        mlflow.log_artifact(history_path, "history")

        # Создание графика истории обучения
        plt.figure(figsize=(10, 6))
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        loss_plot_path = os.path.join(temp_dir, "loss_history.png")
        plt.savefig(loss_plot_path)
        mlflow.log_artifact(loss_plot_path, "plots")
        plt.close()

        # Сохранение  модели в файл
        model.save(f'repo-keras/{run_name}.keras')

        # Логирование модели с сигнатурой
        signature = infer_signature(X_test_scaled, y_pred)
        mlflow.tensorflow.log_model(
            model,
            "keras-model",
            signature=signature,
            registered_model_name={run_name}
        )


In [25]:
dagshub.init(repo_owner='sever.cpa.general', repo_name='my-first-repo', mlflow=True)
experiment_name = "Water Probability [RF]"
run_name = "ml_baseline"
from mlflow.exceptions import MlflowException

try:
    experiment_id = mlflow.create_experiment(experiment_name)
except MlflowException:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

# Основной блок выполнения
# Подготовка данных
X, y, feature_names = prepare_data('..\\WaterQuality\\data\\raw\\water_potability.csv')

# Обучение и оценка модели
results = train_and_evaluate_model(X, y)

# Логирование результатов в MLflow
log_to_mlflow(
    evaluation_results=results,
    experiment_name=experiment_name,  # Используем тот же experiment_name
    run_name=run_name  # Явно передаем run_name
)



Initialized MLflow to track repo "sever.cpa.general/my-first-repo"

Repository sever.cpa.general/my-first-repo initialized!

Epoch 1/50
66/66 - 2s - 36ms/step - accuracy: 0.6054 - loss: 0.6847 - val_accuracy: 0.6050 - val_loss: 0.6777
Epoch 2/50
66/66 - 0s - 5ms/step - accuracy: 0.6054 - loss: 0.6762 - val_accuracy: 0.6050 - val_loss: 0.6723
Epoch 3/50
66/66 - 0s - 6ms/step - accuracy: 0.6054 - loss: 0.6736 - val_accuracy: 0.6050 - val_loss: 0.6714
Epoch 4/50
66/66 - 1s - 11ms/step - accuracy: 0.6054 - loss: 0.6707 - val_accuracy: 0.6050 - val_loss: 0.6710
Epoch 5/50
66/66 - 1s - 9ms/step - accuracy: 0.6054 - loss: 0.6719 - val_accuracy: 0.6050 - val_loss: 0.6710
Epoch 6/50
66/66 - 1s - 9ms/step - accuracy: 0.6054 - loss: 0.6734 - val_accuracy: 0.6050 - val_loss: 0.6711
Epoch 7/50
66/66 - 0s - 7ms/step - accuracy: 0.6054 - loss: 0.6731 - val_accuracy: 0.6050 - val_loss: 0.6712
Epoch 8/50
66/66 - 0s - 5ms/step - accuracy: 0.6054 - loss: 0.6715 - val_accuracy: 0.6050 - val_loss: 0.6710
Epoch 9/50
66/66 - 0s - 4ms/step - accuracy: 0.6054 - loss: 0.6731 - val_accuracy: 0.6050 - val_loss: 0.6711
Epoch 10/50
66/66

Registered model 'binary_classification_keras' already exists. Creating a new version of this model...
2024/12/07 19:46:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: binary_classification_keras, version 10
Created version '10' of model 'binary_classification_keras'.


🏃 View run ml_baseline at: https://dagshub.com/sever.cpa.general/my-first-repo.mlflow/#/experiments/0/runs/d48ae61b7a284e08bb687e301af5a53d
🧪 View experiment at: https://dagshub.com/sever.cpa.general/my-first-repo.mlflow/#/experiments/0
